Esta última tarea se trata de usar todo lo que hemos aprendido en el curso para resolver problemas físicos, por lo que los problemas esta vez no tendrán un orden real. Sólo tendrás que resolver uno de los problemas por tema y el problema final!!

## Problema de mecánica clásica: 

[i] Escribe las ecuaciones de movimiento un péndulo doble. 

[ii] Resuelve para obtener las posiciones de las masas (ambas masas iguales) como función del tiempo. 

[iii] Haz animaciones (del péndulo) para diferentes proporciones de largos de los péndulos. 

## Problema de electromagnetismo: 

[i] Resuelve el modelo de Morris y Lecar para la corriente y voltaje en una neurona. Resolverlo equivale a resolver el siguiente sistema de ecuaciones: 

$$
\begin{align}
  C \frac{dV}{dt} & ~=~ I - g_\mathrm{L} (V-V_\mathrm{L}) - g_\mathrm{Ca} M_\mathrm{ss} (V-V_\mathrm{Ca}) - g_\mathrm{K} N (V-V_\mathrm{K}) \\[5pt]
  \frac{dN}{dt} & ~=~ \frac{N_\mathrm{ss}-N}{\tau_N}
\end{align}
$$

Donde 

$M_{ss} = (1 + \tanh[(V-V_1)/V_2])/2 $ 

$N_{ss} = (1 + \tanh[(V-V_3)/V_4])/2$ 

$\tau_N = 1/[\varphi \cosh[(V-V_3)/(2V_4)]$
 
$V$ es el voltaje y $N$ es la probabilidad de que el canal de iones de potacio en la neurona conduzca. I es la corriente que se inyecta a la neurona. Todo lo demás son constantes. 

[ii] Haz una animación donde grafiques V como función del tiempo para diversas valores de I. (juega un poco con los valores del resto de parámetros). 


In [1]:
using Plots
using Interact 

In [2]:
# Runge Kutta vectorial

function paso_runge_kutta(f, h, t0, x0)
    k1 = f(x0, t0)
    k2 = f(x0 + (h/2)*k1, t0 + h/2)
    k3 = f(x0 + (h/2)*k2, t0 + h/2)
    k4 = f(x0 + h*k3, t0 + h)
    return (x0 + (h/6)*(k1 + 2k2 + 2k3 + k4))
end


function runge_4_vectorial(f, t0, x0, h=0.01; tn=100)
    ts = 0:h:tn
    m_dim = length(x0)
    n_dim = length(ts)
    xs = zeros((m_dim, n_dim))
    xs[:, 1] = x0
    for i in 1:length(ts)-1
        xs[:, i+1] = paso_runge_kutta(f, h, ts[i], xs[:, i])
    end
    return ts, xs
end

runge_4_vectorial (generic function with 2 methods)

In [3]:
function morris_lecar(I, gl=2e-3, gca=4.4e-3, gk=8e-3, Vl=-60e-3, Vk=-80e-3, Vca=120e-3, V1=-1.2e-3, V2=18e-3, V3=12e-3, V4=17.4e-3, C=30e-6, ϕ=1/15)
    Mss(V) = (1 + tanh((V - V1) / V2)) / 2.
    Nss(V) = (1 + tanh((V - V3) / V4)) / 2.
    τN(V) = 1 / (ϕ * cosh((V - V3) / 2*V4))
    sistema(x, t) = [(I - (gl * (x[1] - Vl)) - (gca * Mss(x[1]) * (x[1] - Vca)) - (gk * x[2] * (x[1] - Vk))) / C; (Nss(x[1]) - x[2]) / τN(x[1])]
    t0 = 0
    x0 = [0; 0]
    t, sols = runge_4_vectorial(sistema, t0, x0)
    return t, sols
end

morris_lecar (generic function with 13 methods)

In [4]:
t, sols = morris_lecar(50e-6)

(0.0:0.01:100.0, [0.0 0.0606799 … 0.0155564 0.0155199; 0.0 0.000552202 … 0.388178 0.388319])

In [5]:
anim = @animate for I = 0:1e-6:120e-6
    t, sols = morris_lecar(I)
    plot(t, sols[1, :] / 1e-3, ylims=(-80, 80), ylabel="Voltaje", xlabel="Tiempo")
    end 

gif(anim, "morris_lecar.gif", fps=30)

INFO: Saved animation to /Users/uzielnmtz/FisicaComputacional2018_1/notebooks/morris_lecar.gif


Plots.AnimatedGif("/Users/uzielnmtz/FisicaComputacional2018_1/notebooks/morris_lecar.gif")

Los parámetros empleados en el modelo se obtuvieron de: 

Tsumoto, Kunichika; Kitajimab, Hiroyuki; Yoshinagac, Tetsuya; Aiharad, Kazuyuki; Kawakamif, Hiroshi (January 2006), "Bifurcations in Morris–Lecar neuron model", Neurocomputing (in English and Japanese), 69 (4–6): 293–316, doi:10.1016/j.neucom.2005.03.006

## Problema de mecánica cuántica: 

En este problema resolverás la ecuación de Schödinger usando el algorithmo de Numerov para un potencial arbitrario. 

[i] Escribe la ecuación de Schödinger en una dimensión para un potencial arbitrario en la forma: $\frac{d^2 y}{dx^2} = - g(x) y(x) + s(x) $

[ii] El método de Numerov implica que esta ecuación tiene las 3 soluciones $y_{n-1}, y_n, y_{n+1} $ tomando 3 puntos equidistantes $ x_{n-1}, x_n, x_{n+1} $ que están relacionados como sigue: 

$y_{n+1} \left(1 + \frac{h^2}{12} g_{n+1} \right) = 2 y_n \left(1 - \frac{5 h^2}{12} g_n \right) - y_{n-1} \left(1 + \frac{h^2}{12} g_{n-1} \right) + \frac{h^2}{12} (s_{n+1} + 10 s_n + s_{n-1}) + \mathcal{O} (h^6) $

Utiliza esto para hacer un algoritmo que resuelva tu ecuación. 

[ii] Resuelve la ecuación de Schödingar para un potencial harmónico y comprueba que tu solución coincide con la solución analítica. 


## Problema de física estadística: 

[i] Haz una malla cuadrada 2D de nxn. 

[ii] Resuelve el modelo de Ising esa maya con campo magnético uniforme usando el algoritmo de metrópolis. Es decir, haz una animación de cómo evoluciona el sistema a diferentes temperaturas partiendo de un arreglo desordenado (dibuja cuadros negros cuando el spin sea negativo y blancos cuando sea positivo, o similar).

# EL PROBLEMA FINAL: 

En este punto del curso ya debes tener muy buena idea de qué clase de cosas te llaman más la atención a resolver dentro de la física computacional. 

El problema final de este curso debería entonces ser trivial para ti. 

No requerrirá hacer NADA de talacha, pero sí requiere que te tomes algo de tiempo pensando y también investigando. 

Para evitar que simplemente vayas al final y veas qué es este problema final, no pondré corchetes para anunciarlo. Estará en medio del texto. Leelo con atención. 

En principio ya sabes obtener derivadas, gradientes, integrales, y cualquier operador de cálculo de forma numérica e incluso (usando SymPy) de forma analítica-computacional. Aprendiste a hacer interpolación. Sabes también resolver sistemas de ecuaciones algebráicas lineales. También sabes resolver sistemas de ecuaciones no lineales mediante el método de Newton y el del valor intermedio. Sabes resolver ecuaciones diferenciales en una y varias variables. Aprendiste un poco de ecuaciones diferenciales parciales (ecuación de calor). También aprendiste qué significan condiciones periódicas a la frontera y reflectivas. Aprendiste a resolver problemas de física estadística con métodos de Monte-Carlo. Aprendiste un poco de dinámica Browniana si es que hiciste la práctica. Además aprendiste a graficar bonito, a hacer animaciones y códigos interactivos. Sin embargo, hay aún un sin fin de métodos más que podrías aprender. Por ejemplo, no vimos algoritmos de Monte-Carlo más complejos como el de Wang-Landau, y nos faltó mucho sobre solución a problemas cuánticos(apenas ahora es un problema de la tarea. Tampoco vimos como hacer análisis de caoticidad. Ahora, como último problema tienes que decir qué proyecto final harás y cómo pretendes resolverlo. Este puede ser desarrollár algo de lo que no vimos y resolver algún problema con ello, o resolver un problema complejo con los algoritmos que vimos. Así que a pensar, investigar y preguntar alos profesores sobre la viabilidad del proyecto. Este problema vale la mitad de la tarea. 